### Creating the project

In [ ]:
# create the project:
$ django-admin startproject mysite

# the infrastructure:
mysite/ # name doesn't matter
    manage.py # for  interaction with project
    mysite/
        __init__.py
        settings.py # project settings
        urls.py # "table of content"
        wsgi.py # start for wsgi servers

In [ ]:
# in folder mysite run the server
$ python manage.py runserver 8000
# access it at http://127.0.0.1:8000/

### Creating an app

In [ ]:
# in main folder mysite/ create an app
$ python manage.py startapp polls

# the infrastructure
polls/
    __init__.py
    admin.py
    apps.py
    migrations/
        __init__.py
    models.py
    tests.py
    views.py
    -> urls.py # add later

#### Write the first view

In [ ]:
# edit polls/views.py
from django.http import HttpResponse


def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")
# The simplest view in Django.
# To call it we need to map it to URL.

In [ ]:
# Add "urls.py"  to the polls/ folder
# edit polls/urls.py
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^$', views.index, name='index'),
]

In [ ]:
# Point the root URLconf at polls.urls module
# edit mysite/urls.py
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^polls/', include('polls.urls')),
    url(r'^admin/', admin.site.urls),
]

# Always use include for other URL patterns.
# admin.site.urls is the only exception.

In [ ]:
# Try if the view works.
$ python manage.py runserver 8000
# Access it at http://127.0.0.1:8000/polls/
# You should see the text.

#### Database setup

In [ ]:
# change mysite/settings.py
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'mydatabase',
        'USER': 'mydatabaseuser',
        'PASSWORD': 'mypassword',
        'HOST': '127.0.0.1',
        'PORT': '5432',
    }
}

In [ ]:
# The INSTALLED_APS contains the default apps.
# Some of them need at least one database table, so create it:
$ python manage.py migrate
# It looks at the INSTALLED_APS  and creates neccessaty DBs.
# Check with \dt in postgreSQL.

### Creating models
Essentially the database layout.

In [ ]:
# edit polls/models.py
# Class represents a database table.
# Class attributes represent a table field.
from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

#### Activating model

In [ ]:
# Add the new model myseite/settings.py
INSTALLED_APPS = [
    'polls.apps.PollsConfig',
    ...
]

# Tell django you made changes
$ python manage.py makemigrations polls
# Migrations is how django stores changes to models
# What would that migrate do?
$ python manage.py sqlmigrate polls 0001

# Now run migrate again to create those model tables in DB
$ python manage.py migrate
# It takes all the migrations that you haven't applies and runs them.

In [ ]:
1. # Change your models (in models.py).
2. $ python manage.py makemigrations # to create migrations for those changes
3. $ python manage.py migrate # to apply those changes to the database.

### Playing with API

In [ ]:
# Enter with
$ python manage.py shell

In [ ]:
# Play with the API
from polls.models import Question, Choice   # Import the model classes we just wrote.

Question.objects.all() # List all Question instances

# Create a question (a row in a table)
from django.utils import timezone
q = Question(question_text="What's new?", pub_date=timezone.now())
q.save() # Save into database!

# Call its attributes
q.id
q.question_text
# Change values
q.question_text = "What's up?"
q.save()

### Create an admin user and admin pagr

In [ ]:
# Create admin user.
$ python manage.py createsuperuser

In [ ]:
# Enter the admin site at http://127.0.0.1:8000/admin/
$ python manage.py runserver 8000

In [ ]:
# Make the poll app modifiable in admin.
# edit polls/admin.py
from django.contrib import admin

from .models import Question

admin.site.register(Question)

# Now you have the options to add, remove and edit questions.